In [4]:
user_queries = [
        "장기보험상품에서 암진단 후 지급기간은?",
        "장기보험상품에서 보장주기에 대해 알려줘",
        "장기보험상품에서 보장범위는 어떻게 되나요?", 
        "장기보험상품에서 보장 불가한 항목은 어떻게 되나요?",
        "장기보험상품에서 보장 가능한 병원등급은 어떻게 되나요?",
        "장기보험상품에서 보장하는 방식에서 몇 회까지 보장하는지 알려줘",
        "장기보험상품에서 보장하는 기준을 알려줘",
]

In [1]:
from openai import OpenAI

model = "qwen3:8b"
#일반보험_챗봇 id
# client = OpenAI(
#     api_key="ragflow-U5ZGEyNTdlNjkyODExZjBiODE2MDI0Mm", 
#     base_url=f"http://10.50.7.154:8080/api/v1/chats_openai/d0f38fb26c1811f0bbf00242ac150007"
# )
#장기보험_챗봇 id: 269d722c6c3a11f097260242ac150007
client = OpenAI(
    api_key="ragflow-U5ZGEyNTdlNjkyODExZjBiODE2MDI0Mm", 
    base_url=f"http://10.50.7.154:8080/api/v1/chats_openai/269d722c6c3a11f097260242ac150007"
)

def retrieve_and_generate(question, stream=False):
    completion = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "user", "content": question},
        ],
        stream=stream,
    )

    if stream:
        full_response = ""
        for chunk in completion:
            # Extract the content from the chunk
            if hasattr(chunk.choices[0].delta, "content") and chunk.choices[0].delta.content:
                content = chunk.choices[0].delta.content
                full_response += content
                # Replace \n and \n\n with actual line breaks and print
                print(content.replace("\\n", "\n"), end="")
        print("\n\nFinal Processed Response:")
        print(full_response.replace("\\n", "\n")) 
    else:
        content = completion.choices[0].message.content
        print(content)

In [5]:
retrieve_and_generate(user_queries[0], stream=False)

<think>
Okay, the user is asking about the payment period for cancer diagnosis in a long-term insurance product. Let me check the knowledge base provided.

Looking at the documents, there are several entries related to cancer diagnosis and payment terms. The first document has entries ID 0, 1, 2, 3, 4, 5, 6. Entry ID 0 mentions a 90-day waiting period for cancer diagnosis, and after that, there's a possible reduction in benefits within the first year. Entry ID 3 repeats similar information about the 90-day waiting period and potential benefit reduction. Entry ID 6 talks about payment periods based on when the diagnosis occurs relative to the contract date. For example, if diagnosed within one year of the contract date, payments are 50% of the policy amount over 20 installments. If after one year, it's full amount. 

Another entry, ID 5, mentions that once the cancer diagnosis occurs, the special policy terminates and doesn't pay the accumulated amount. But this might be about the termi

In [6]:
from langfuse import Langfuse

langfuse = Langfuse(
  secret_key="sk-lf-8b2404f1-6a48-4dfb-b62e-5db0cc0f28c8",
  public_key="pk-lf-3cf05432-2a74-47f0-bf6c-fb5859c5cb50",
  host="http://10.50.3.31:3000"
)

Failed to export span batch code: 401, reason: {"extra":null,"message":"Invalid username or password","messageId":"password-auth.failed","statusCode":401,"traceID":""}



In [7]:
from langfuse.openai import OpenAI
from langchain_core.output_parsers import StrOutputParser

#장기보험_챗봇 id: 269d722c6c3a11f097260242ac150007
client = OpenAI(
	api_key="ragflow-U5ZGEyNTdlNjkyODExZjBiODE2MDI0Mm", 
    base_url=f"http://10.50.7.154:8080/api/v1/chats_openai/269d722c6c3a11f097260242ac150007"
)

def retrieve_and_generate(question, stream=False):
    completion = client.chat.completions.create(
        model="qwen3:8b",
        messages=[
            {"role": "user", "content": question},
        ],
        stream=stream,
    )
    output_parser = StrOutputParser()

    if stream:
        full_response = ""
        for chunk in completion:
            # Extract the content from the chunk
            if hasattr(chunk.choices[0].delta, "content") and chunk.choices[0].delta.content:
                content = chunk.choices[0].delta.content
                output = output_parser.parse(content)
                # 파싱된 결과 출력
                print(output, end="")
                full_response += content
        #output = output_parser.parse(full_response)
        #print(output)
    else:
        content = completion.choices[0].message.content
        output = output_parser.parse(content)
        print(output)

In [8]:
retrieve_and_generate(user_queries[1], stream=True)

<think>
Okay, the user is asking about the coverage period for long-term insurance products. Let me check the knowledge base provided.

Looking at the documents, there's a mention of renewal periods. In document 2B, IDs 1 and 2 talk about selecting a 20-year or 30-year renewable term with no refund upon cancellation. That's important for understanding the coverage duration. Also, ID 3 mentions that when contracting for 20 or 30-year renewable terms (without refund), the company compares premiums and refund rates. 

Additionally, document 2A has information about risk reduction over time and reduced payouts if claims occur within a certain period. While this is more about risk assessment, itit relates to how coverage might adjust over time. However, the user specifically asked aboutt the coverage period, so the main points are the renewable terms and their implications.

I need to structure the answer around the renewable periods (20 or 30 years), mention that refunds aren't given upon 

In [9]:
from ragflow_sdk import RAGFlow

api_key = "ragflow-U5ZGEyNTdlNjkyODExZjBiODE2MDI0Mm"
base_url = "http://10.50.7.154:8080"

rag_object = RAGFlow(api_key=api_key, base_url=base_url)

# 모든 지식베이스 조회
dataset_ids = []
for dataset in rag_object.list_datasets(name="장기보험상품"):
    print(dataset)
    dataset_ids.append(dataset.id)

chunks = rag_object.retrieve(dataset_ids=dataset_ids, question=user_queries[0])
print(f'len: {len(chunks)}')
for c in chunks:
    print(f'{c.document_name}-{len(c.content)}: {c.content}')
# datasets = rag_object.list_datasets(name="장기보험상품")
# dataset_id = datasets[0].id
# print(dataset_id)
# rag_object.retrieve(dataset_ids=[dataset_id], question=user_queries[0])

{'avatar': None, 'chunk_count': 22022, 'chunk_method': 'naive', 'description': '장기보험 상품약관, 상품요약서, 사업방법서', 'document_count': 3, 'embedding_model': 'bona/bge-m3-korean:latest', 'id': 'ad61502219184da99b4e4ea453425e49', 'name': '장기보험상품', 'pagerank': 0, 'parser_config': {'auto_keywords': 0, 'auto_questions': 0, 'chunk_token_num': 512, 'delimiter': '\n!?;。；！？', 'graphrag': {'use_graphrag': False}, 'html4excel': False, 'layout_recognize': 'MinerU', 'raptor': {'use_raptor': False}}, 'permission': 'team', 'tenant_id': 'ca5b27a4499e4db7aa9300ba721d344e'}
len: 30
-175: 약관에 의거하여 암(갑상선암, 제자리암, 기타피부암 및 경계성종양 제외)은 계약일 이후 90일의면책기간을 설정하고 있으며, 면책기간에 발생한 암진단(갑상선암, 제자리암, 기타피부암 및 경계성종양 제외)은 보험금을 지급하지 않습니다. 90일이 경과된 이후에도 일정기간(1년) 이내인 경우보험금이 삭감될 수 있습니다.
-247: 1. 암진단(유사암제외)정기검진비(10년)(6개월지급형) : 이 특별약관의 보험기간 중에 제3조(보험금 지급에 관한 세부규정) 제1항에서 정한 보장개시일 이후에 제2조(「암(유사암제외)」의 정의 및 진단 확정)의 「암(유사암제외)」으로 진단이 확정되었을 때에는 최초 1회에 한하여 아래의 금액을 지급사유가 발생한 날부터 10년동안 매6개월마다 지급사유 발생일(단, 해당월에 보험금 지급사유 발생일이 없는 경우해당월의 마지막날로 합니다)에 지급
-143

In [ ]:
from ragflow_sdk import RAGFlow

api_key = "ragflow-U5ZGEyNTdlNjkyODExZjBiODE2MDI0Mm"
base_url = "http://10.50.7.154"
rag_object = RAGFlow(api_key=api_key, base_url=f"{base_url}:9380")
assistant = rag_object.list_chats(name="장기보험_챗봇")
assistant = assistant[0]
session = assistant.create_session()    

print("\n==================== 장기보험 챗봇 =====================\n")
print("Hello. What can I do for you?")

while True:
    question = input("\n==================== User =====================\n> ")
    print("\n==================== 장기보험 챗봇 =====================\n")
    
    cont = ""
    for ans in session.ask(question, stream=True):
        print(ans.content[len(cont):], end='', flush=True)
        cont = ans.content